# Manually define outlets

In [1]:
import sys
import pandas as pd
from pathlib import Path
sys.path.append(str(Path().absolute().parent))
import python_cs_functions as cs

### Config handling

In [2]:
# Specify where the config file can be found
config_file = '../0_config/config.txt'

In [3]:
# Get the required info from the config file
data_path     = cs.read_from_config(config_file,'data_path')
cs_meta_path  = cs.read_from_config(config_file,'cs_basin_path')
cs_meta_name  = cs.read_from_config(config_file,'cs_meta_name')

### Load the metadata

In [4]:
# CAMELS-spat metadata file
cs_meta_path = Path(data_path) / cs_meta_path
cs_meta = pd.read_csv(cs_meta_path / cs_meta_name)

### Update the basin outlet locations

In [5]:
# Add a Delineation_notes column
cs_meta['Delineation_notes'] = 'n/a'

In [6]:
# Define the changes we need to make
# The will go in a list that we will later integrate into the metadata dataframe
# [country, gauge, new lon, new lat, description]
outlet_changes = []
outlet_changes.append(['CAN','01AD002', -68.59414, 47.25770, 'moved outlet to middle of wide river'])


In [7]:
# Ensure we have the correct column IDs
c_lat, c_lon, c_man, c_dsc = 6,7,8,17
assert (cs_meta.columns[c_lat] == 'Outlet_lat') # If these aren't true, they'll pop an error
assert (cs_meta.columns[c_lon] == 'Outlet_lon')
assert (cs_meta.columns[c_man] == 'Outlet_source')
assert (cs_meta.columns[c_dsc] == 'Delineation_notes')

# Apply the changes to the metadata file
for change in outlet_changes:
    
    country = change[0]
    station = change[1]
    lon = change[2]
    lat = change[3]
    dsc = change[4]
    
    idx = cs_meta.index[(cs_meta['Country'] == country) & (cs_meta['Station_id'] == station)][0] # There can be only one
    
    cs_meta.iat[idx,c_lat] = lat
    cs_meta.iat[idx,c_lon] = lon
    cs_meta.iat[idx,c_man] = 'Manual'
    cs_meta.iat[idx,c_dsc] = dsc
    

In [8]:
cs_meta

,Country,Station_id,Station_name,Station_lat,Station_lon,Station_source,Outlet_lat,Outlet_lon,Outlet_source,Basin_area_km2,Ref_area_1_src,Ref_area_1_km2,Ref_area_2_src,Ref_area_2_km2,Ref_shape,Ref_shape_source,Ref_shape_area_km2,Delineation_notes
0,CAN,01AD002,SAINT JOHN RIVER AT FORT KENT,47.25806,-68.59583,WSC 20222 data set,47.257700,-68.594140,Manual,-999,HYDAT gross drainage area,14700.00,HYDAT effective drainage area,NaN,yes,WSC 2022 data set,14677.400000,moved outlet to middle of wide river
1,CAN,01AD003,ST. FRANCIS RIVER AT OUTLET OF GLASIER LAKE,47.20661,-68.95694,WSC 20222 data set,47.206016,-68.956727,WSC 20222 data set,-999,HYDAT gross drainage area,1350.00,HYDAT effective drainage area,NaN,yes,WSC 2022 data set,1348.300000,n/a
2,CAN,01AE001,FISH RIVER NEAR FORT KENT,47.23750,-68.58278,WSC 20222 data set,47.237438,-68.581991,WSC 20222 data set,-999,HYDAT gross drainage area,2260.00,HYDAT effective drainage area,NaN,yes,WSC 2022 data set,2238.950000,n/a
3,CAN,01AF007,GRANDE RIVIERE AT VIOLETTE BRIDGE,47.24697,-67.92128,WSC 20222 data set,47.246908,-67.921271,WSC 20222 data set,-999,HYDAT gross drainage area,339.00,HYDAT effective drainage area,NaN,yes,WSC 2022 data set,325.196000,n/a
4,CAN,01AF009,IROQUOIS RIVER AT MOULIN MORNEAULT,47.45783,-68.35683,WSC 20222 data set,47.457622,-68.356710,WSC 20222 data set,-999,HYDAT gross drainage area,182.00,HYDAT effective drainage area,NaN,yes,WSC 2022 data set,183.430000,n/a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1693,USA,14309500,"WEST FORK COW CREEK NEAR GLENDALE, OR",42.80400,-123.61091,CAMELS-US,-999.000000,-999.000000,NaN,-999,GAGES II area,224.92,Geospatial fabric area,226.31,yes,CAMELS-US data set (HCDN),226.314346,n/a
1694,USA,14316700,"STEAMBOAT CREEK NEAR GLIDE, OR",43.34984,-122.72894,CAMELS-US,-999.000000,-999.000000,NaN,-999,GAGES II area,587.90,Geospatial fabric area,588.01,yes,CAMELS-US data set (HCDN),588.025033,n/a
1695,USA,14325000,"SOUTH FORK COQUILLE RIVER AT POWERS, OR",42.89150,-124.07065,CAMELS-US,-999.000000,-999.000000,NaN,-999,GAGES II area,443.07,Geospatial fabric area,444.92,yes,CAMELS-US data set (HCDN),444.925651,n/a
1696,USA,14362250,"STAR GULCH NEAR RUCH, OR.",42.15401,-123.07532,CAMELS-US,-999.000000,-999.000000,NaN,-999,GAGES II area,41.42,Geospatial fabric area,43.88,yes,CAMELS-US data set (HCDN),43.877901,n/a


In [9]:
# Save the updated metadata file
cs_meta.to_csv(cs_meta_path / cs_meta_name, encoding='utf-8', index=False)